# A PyNets Primer in Python and Bash

## Scope of this tutorial
Docker/Singularity containers are the preferable way to run PyNets because the compute environment will include all optional dependencies and will yield fully reproducible numerical precision. To keep things simple for this demonstration, however, let's begin by just installing PyNets in a virtual environment and then run the workflow manually on some example BIDS data. The scope of this tutorial will cover single-subject workflows. For more examples (i.e. including usage with docker/singularity), see: https://pynets.readthedocs.io/en/latest/usage.html

## What PyNets is and isn't
Although we will explore the package interactively in the code that follows, keep in mind that PyNets is not a library like its core dependencies (Nipype, Dipy, Nilearn, Networkx). Rather, it is a *workflow* for mining connectome ensembles that uses these libraries in an integrated fashion. More specifically, PyNets is a *post-processing* workflow, which means that it expects dMRI and fMRI data inputs that have already been scrubbed of artifact. This helps ensure that analytically-relevant resampling (e.g. normalization to a template, smoothing, high/low-pass filtering, ROI masking, etc.) can be left to PyNets for connectome optimization, whilst also providing flexibility to the user to test their own unique hypotheses (e.g. the impact of various fmri-denoising strategies on downstream connectome reliability). fmriprep, CPAC, XCP-Engine, dmriprep, qsiprep, etc. are ideal preprocessing pipelines to use in this regard because they operate on BIDS datasets more or less automatically, and thus can easily scale to larger sample-sizes needed to minimize prediction bias.

## Scalability and compute resources
PyNets itself was developed primarily using supercomputers and cloud infrastructures, but it is flexible enough to also run using just a few cores on your local laptop/desktop. This tutorial is meant to demonstrate what PyNets makes possible in <2 hours. Welcome to the exciting new world of connectome ensemble-learning.

## Installation

In [1]:
%%bash
# Assuming that python3, pip, and FSL are already installed...
# Start a virtual environment and install some dependencies for our lesson.
pip install virtualenv --user
mkdir ~/virtualenvironment 2>/dev/null
virtualenv ~/virtualenvironment/pynets
cd ~/virtualenvironment/pynets/bin
source activate
./pip3 install -U gdown fury ipykernel # for downloading data, running pynets, and some 3d viz
./pip3 install pynets=='1.0.7'

Using base prefix '/usr/local/anaconda3'
New python executable in /Users/derekpisner/virtualenvironment/pynets/bin/python
Installing setuptools, pip, wheel...
done.
Requirement already up-to-date: gdown in /Users/derekpisner/virtualenvironment/pynets/lib/python3.7/site-packages (3.12.0)
Requirement already up-to-date: fury in /Users/derekpisner/virtualenvironment/pynets/lib/python3.7/site-packages (0.6.0)
Requirement already up-to-date: ipykernel in /Users/derekpisner/virtualenvironment/pynets/lib/python3.7/site-packages (5.3.4)
  Attempting uninstall: pynets
    Found existing installation: pynets 1.0.6
    Uninstalling pynets-1.0.6:
      Successfully uninstalled pynets-1.0.6


In [2]:
%%bash
# Any GUI-based visualizer of overlays will do for QC, but for demonstration purposes,
# I will use FSLeyes due to its user-ease. Future PyNets versions will
# include html-style reports (ala fmriprep/qsiprep/dmriprep).
# For macs, download this link for fsleyes:
if  [ ! -f /tmp/FSLeyes-latest-macos.tar.gz ] || [ ! -d ~/virtualenvironment/FSLeyes.app ]; then
    wget https://fsl.fmrib.ox.ac.uk/fsldownloads/fsleyes/FSLeyes-latest-macos.tar.gz -O /tmp/FSLeyes-latest-macos.tar.gz
    tar -xzf /tmp/FSLeyes-latest-macos.tar.gz -C ~/virtualenvironment
fi
rm /tmp/FSLeyes-latest-macos.tar.gz
# For Linux, downnload the appropriate precompiled build from https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FSLeyes

--2020-07-31 16:08:32--  https://fsl.fmrib.ox.ac.uk/fsldownloads/fsleyes/FSLeyes-latest-macos.tar.gz
Resolving fsl.fmrib.ox.ac.uk (fsl.fmrib.ox.ac.uk)... 129.67.248.65
Connecting to fsl.fmrib.ox.ac.uk (fsl.fmrib.ox.ac.uk)|129.67.248.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 378058752 (361M) [application/x-gzip]
Saving to: ‘/tmp/FSLeyes-latest-macos.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  207K 29m46s
    50K .......... .......... .......... .......... ..........  0%  416K 22m17s
   100K .......... .......... .......... .......... ..........  0% 43.3M 14m54s
   150K .......... .......... .......... .......... ..........  0%  430K 14m45s
   200K .......... .......... .......... .......... ..........  0% 19.1M 11m51s
   250K .......... .......... .......... .......... ..........  0% 78.1M 9m54s
   300K .......... .......... .......... .......... ..........  0%  432K 10m31s
   350K .......... .......... .......... .....

# Fetch sample preprocessed data

Now we can download a minimal dataset from OASIS that includes preprocessed, multimodal fMRI and dMRI data.

*Note*: Normally, we could just use a dataset from datalad or from s3 (which will download automatically using an s3:// file path prefix for all file path inputs to `pynets_bids` or `pynets_cloud` if your AWS credentials are properly configured!). See examples here: https://pynets.readthedocs.io/en/latest/usage.html#quickstart

In [3]:
%%bash
# Now we create an output directory for the derivatives of the pipeline (if one doesn't exist already).
if  [ ! -d ~/Downloads/.pynets ]; then
    mkdir ~/Downloads/.pynets
fi

# And download the data to a generic "derivatives" directory.
if  [ ! -f ~/Downloads/.pynets/test_oasis.tar.gz ]; then
    cd ~/Downloads/.pynets
    gdown https://drive.google.com/uc?id=1beEoc_Pdk6OBDYc80mBDTvUhcUny9Gu3 -O ~/Downloads/.pynets/test_oasis.tar.gz
else
    cd ~/Downloads/.pynets
fi

mkdir -p ~/Downloads/.pynets/derivatives/preprocessed/sub-OAS31172 2>/dev/null
tar -xzf test_oasis.tar.gz -C derivatives/preprocessed/sub-OAS31172
rm test_oasis.tar.gz

Downloading...
From: https://drive.google.com/uc?id=1beEoc_Pdk6OBDYc80mBDTvUhcUny9Gu3
To: /Users/derekpisner/Downloads/.pynets/test_oasis.tar.gz
230MB [00:05, 40.4MB/s]


Now we prepare PyNets to run o a single subject from the data we just downloaded. We can do this in two ways -- (1) using the `pynets_bids` API since our sample data is in BIDS format and can be queried using pybids; (2) with the `pynets` API for comparison. We we go with the first option (1), which requires that we specify a connectome recipe or recipees in the form of a .json config file. For comparison, purposes, equivalent command-line call for option (2) is also prvided and commented out.

So, for run 1 of session d0407 from subject OAS31172, lets sample an ensemble of 144 functional connectome estimates (1 models x 6 thresholds x 2 smoothing values x 2 high-pass filter thresholds x 3 atlases x 2 time-series extraction methods).

In [6]:
%%bash
# Config file
cat ~/virtualenvironment/pynets/lib/python3.7/site-packages/pynets/config/bids_config_bold.json

{
    "dwi": {
            "dg": "None",
            "ml": "None",
            "mod": "None"
        },
    "func": {
            "ct": "None",
            "hp": "['0', '0.028']",
            "k": "None",
            "mod": "['partcorr']",
            "sm": "['0', '4']",
            "es": "['mean', 'variance']"
        },
    "gen": {
            "a":  "['BrainnetomeAtlasFan2016', 'atlas_harvard_oxford', 'destrieux2009_rois']",
            "bin":  "False",
            "df":  "False",
            "dt":  "False",
            "embed":  "True",
            "max_thr":  0.80,
            "min_thr":  0.20,
            "mplx":  "['0']",
            "mst":  "True",
            "n":  "None",
            "norm": "['6']",
            "ns":  "None",
            "p":  "['1']",
            "plt":  "True",
            "spheres":  "False",
            "step_thr":  0.10,
            "thr":  "None",
            "vox":  "'2mm'"
        }
}


In [ ]:
%%bash
dir=~/Downloads/.pynets
abs_dir=`echo "$(dirname $dir)"`

# BIDS way using a pre-configured .json file that specifies how we want the pipeline to run.
# We can view this file to get an idea of what it contains:
cat ~/virtualenvironment/pynets/lib/python3.7/site-packages/pynets/config/bids_config_bold.json

# Next we initiate the `pynets_bids` CLI (note the inclusion of a run_label since the BOLD acquisitions for this dataset contain two runs):%%bash
~/virtualenvironment/pynets/bin/pynets_bids "$abs_dir"/.pynets/derivatives/preprocessed "$abs_dir"/.pynets/derivatives participant func --participant_label OAS31172 --session_label d0407 --run_label 1 -config ~/virtualenvironment/pynets/lib/python3.7/site-packages/pynets/config/bids_config_bold.json

# *Note that the configuration in `bids_config_bold.json` is equivalent to running the following (non-BIDS) CLI call that does not require a config file:
# ~/virtualenvironment/pynets/bin/pynets "$abs_dir"/.pynets/derivatives -id OAS31172_d0407_1 -mod 'partcorr' -min_thr 0.20 -max_thr 0.80 -step_thr 0.10 -sm 0 4 -hp 0 0.028 -a 'BrainnetomeAtlasFan2016' 'atlas_harvard_oxford' 'destrieux2009_rois' -es 'mean' 'variance' -anat "$abs_dir"/.pynets/derivatives/sub-OAS31172/ses-d0407/anat/sub-OAS31172_ses-d0407_run-01_T1w.nii.gz -func "$abs_dir"/.pynets/derivatives/sub-OAS31172/ses-d0407/func/sub-OAS31172_ses-d0407_task-rest_run-01_bold.nii.gz -plug 'MultiProc' -work '/tmp/pynets_work' -mst -plt -embed

# Viewing outputs

In [ ]:
%%bash
cd ~/Downloads/.pynets/derivatives/pynets/sub-OAS31172/ses-d0407/func
ls

tree

Lets do a bit of quality-control to ensure, for example, that the inverse warping of the destrieux2009_rois atlas from template-space to native T1w anatomical space is valid.

In [ ]:
%%bash
t1w_image=`ls /tmp/pynets_work/*/*/meta_wf_*/fmri_connectometry*/register_node/reg/imgs/*t1w_brain.nii.gz | head -1`
atlas_in_t1w_image=`ls /tmp/pynets_work/*_wf_single_subject_fmri*/wf_single_*/meta_wf_*/fmri_connectometry_*/_atlas_destrieux2009_rois/register_atlas_node/atlas_destrieux2009_rois/*_gm.nii.gz | head -1`

#~/virtualenvironment/FSLeyes.app/Contents/*/fsleyes "$t1w_image" "$atlas_in_t1w_image" -cm 'random' &


In [ ]:
import glob
from pathlib import Path
from IPython.display import Image
Image(filename=glob.glob(str(Path('~').expanduser()) + '/Downloads/.pynets/derivatives/pynets/sub-OAS31172/ses-d0407/func/BrainnetomeAtlasFan2016/figures/glassbrain_*.png')[0])


Above is a glass brain depiction of bilateral regions of the Brainnetome atlas (Fan et al., 2016) using a partial correlation estimator, 4 fwhm smoothing, 0.028Hz high-pass filter, based on variance of the node-extracted time-series, with 20% post-hoc thresholding using the Minimum-Spanning Tree (MST) method. The latter method serves as an anti-fragmentation device that ensures we can prevent isolated (i.e. disconnected) nodes that can violate certain graph theoretical assumptions.

In the visualization, node size conveys the level of node importance (smaller is lower eigenvector centrality) and node color corresponds to hierarchical Louvain community affiliation (8 distinct communities found).

The below adjacency matrix depicts a single connectome estimate, with community affiliation.

In [ ]:
Image(filename=glob.glob(str(Path('~').expanduser()) + '/Downloads/.pynets/derivatives/pynets/sub-OAS31172/ses-d0407/func/BrainnetomeAtlasFan2016/figures/*')[0])

We could also look at the mean connectome (i.e. across all 144 estimates) -- what we might from here on out refer to as an **omnetome** as well:

In [ ]:
%matplotlib inline
import glob
import numpy as np
from matplotlib import pyplot as plt
from nilearn.plotting import plot_matrix
from pynets.core.thresholding import standardize, binarize

mats = [np.load(i) for i in glob.glob('/Users/*/Downloads/.pynets/derivatives/pynets/sub-OAS31172/ses-d0407/func/BrainnetomeAtlasFan2016/graphs/*.npy')]

mean_mat = standardize(np.mean(mats, axis=0))

mean_mat_bin = binarize(mean_mat)
mean_mat = np.nan_to_num(np.multiply(mean_mat, mean_mat_bin))

plot_matrix(
    mean_mat,
    figure=(10, 10),
    labels=[' ']*len(mean_mat),
    vmax=np.percentile(mean_mat[mean_mat > 0], 95),
    vmin=np.percentile(mean_mat[mean_mat > 0], 5),
    reorder="average",
    auto_fit=True,
    grid=False,
    colorbar=True,
    cmap='RdBu',
)
plt.show()

As you can see, we get a much more information-rich graph. More specifically, there is more well-defined community structure and there is more variance. This graph, unlike the first, now represents a *distribution* of connectomes, that, by virtue of its plurality of views, more exhaustively samples from the true *population* of possible brain network configurations in this individual that may exhibit connectivity, across the whole brain as a region of interest, at any point in time during the course of the 5-10 minute resting-state time-series.

# Collecting Outputs
So, we explored the outputs of our connectome ensemble visually, but let's take a closer look at our omnetome's topology. To do this, we run another workflow using the `pynets_collect` CLI, which collects the various graph topological metrics extracted from each of the connectome estimates in our ensemble.

In [ ]:
%%bash

dir=~/Downloads/.pynets
abs_dir=`echo "$(dirname $dir)"`
~/virtualenvironment/pynets/bin/pynets_collect -basedir "$abs_dir"/.pynets/derivatives/pynets -modality 'func'

In [ ]:
import pathlib
from pathlib import Path
import pandas as pd

# Now we can load a dataframe containing all the AUC topological graph metrics calculated for this particular subject's run:
p = str(Path('~').expanduser()) + '/Downloads/.pynets/derivatives/pynets/func_group_topology_auc/sub-OAS31172_ses-d0407_topology_auc_clean.csv'
df_individual = pd.read_csv(p, index_col=False)
df_individual

# Note that if we were to sample connectomes from multiple subjects, the previous pynets_collect CLI would
# simply append new rows to the summary `all_subs_neat_func.csv` dataframe.
p = str(Path('~').expanduser()) + '/Downloads/.pynets/derivatives/pynets/all_subs_neat_func.csv'
df_group = pd.read_csv(p, index_col=False)
df_group

The below multiplot depicts distributions of average graph topological metrics, calculated using Area-Under-the-Curve (AUC) across our window of multiple thresholds, for the ensemble of 144 connectomes sampled. As you can visually discern, topology varies considerably across estimates.

In [ ]:
%matplotlib inline
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
from pynets.plotting.plot_gen import plot_graph_measure_hists

csv_all_metrics = str(Path('~').expanduser()) + '/Downloads/.pynets/derivatives/pynets/func_group_topology_auc/sub-OAS31172_ses-d0407_topology_auc_clean.csv'

out = plot_graph_measure_hists(csv_all_metrics)
out.show()

We could also plot the omnetome embeddings to visualize lower-dimensional latent positions of the ensemble corresponding to each distinct graph resolution

In [ ]:
%matplotlib inline
from graspy.plot import pairplot
import glob
import os
import numpy as np
from matplotlib import pyplot as plt

omnetomes = [[np.load(i), os.path.basename(i).split('.npy')[0]] for i in glob.glob('/Users/*/Downloads/.pynets/derivatives/pynets/sub-OAS31172/ses-d0407/func/*/embeddings/*omnetome.npy')]

for grad, title in omnetomes:
    plot = pairplot(grad, title=title + '_Functional')
plt.show()

# Structural Connectometry

In [ ]:
%%bash
%%capture

# Again, get the absolute paths to files and directories we will use.
# The CLI's in PyNets do NOT accept relative paths.
dir=~/Downloads/.pynets
abs_dir=`echo "$(dirname $dir)"`

# BIDS way using a pre-configured .json file that specifies how we want the pipeline to run.
# We can view this file to get an idea of what it contains:
cat ~/virtualenvironment/pynets/lib/python3.7/site-packages/pynets/config/bids_config_dwi.json

# Next we initiate the `pynets_bids` CLI:
~/virtualenvironment/pynets/bin/pynets_bids "$abs_dir"/.pynets/derivatives/preprocessed "$abs_dir"/.pynets/derivatives participant dwi --participant_label OAS31172 --session_label d0407 -config ~/virtualenvironment/pynets/lib/python3.7/site-packages/pynets/config/bids_config_dwi.json

# *Note that the configuration in `bids_config_dwi.json` is equivalent to running the following (non-BIDS) CLI call that does not require a config file:
#~/virtualenvironment/pynets/bin/pynets "$abs_dir"/.pynets/outputs -mod 'csa' -min_thr 0.20 -max_thr 0.80 -step_thr 0.10 -dg 'det' 'prob' -ml 20 0 -a 'BrainnetomeAtlasFan2016' 'atlas_harvard_oxford' 'destrieux2009_rois' -anat ""$abs_dir"/.pynets/derivatives/sub-OAS31172/ses-d0407/anat/sub-OAS31172_ses-d0407_run-01_T1w.nii.gz" -dwi ""$abs_dir"/.pynets/derivatives/sub-OAS31172/ses-d0407/dwi/sub-OAS31172_ses-d0407_dwi.nii.gz" -bval ""$abs_dir"/.pynets/derivatives/sub-OAS31172/ses-d0407/dwi/sub-OAS31172_ses-d0407_dwi.bval" -bvec ""$abs_dir"/.pynets/derivatives/sub-OAS31172/ses-d0407/dwi/sub-OAS31172_ses-d0407_dwi.bvec" -id OAS31172_d0407_1 -plug 'MultiProc' -work '/tmp/pynets_work' -mst -plt -vox '2mm' -embed


# Viewing outputs

In [ ]:
%%bash
cd ~/Downloads/.pynets/derivatives/pynets/sub-OAS31172/ses-d0407/dwi
ls
tree

Lets do a bit of quality-control to ensure, for example, that the inverse warping of the harvard_oxford atlas from template-space to native DWI anatomical space is valid.

In [ ]:
%%bash
t1w_dwi_image=`ls /tmp/work/*/*/meta_wf_*/dmri_connectometry*/register_node/dmri_reg/reg/imgs/t1w_in_dwi.nii.gz | head -1`
atlas_in_t1w_dwi_image=`ls ~/Downloads/.pynets/outputs/pynets/sub-OAS31172/ses-d0407/dwi/atlas_harvard_oxford/parcellations/*_dwi_track.nii.gz | head -1`
density_map=`ls ~/Downloads/.pynets/outputs/pynets/sub-OAS31172/ses-d0407/dwi/atlas_harvard_oxford/tractography/*.nii.gz | head -1`

#~/virtualenvironment/FSLeyes.app/Contents/*/fsleyes "$t1w_dwi_image" "$atlas_in_t1w_dwi_image" -cm 'random' "$density_map" -cm 'copper' &


In [ ]:
import glob
from IPython.display import Image
Image(filename=glob.glob('/Users/*/Downloads/.pynets/derivatives/pynets/sub-OAS31172/ses-d0407/dwi/atlas_harvard_oxford/figures/glassbrain_OAS31172_d0407_modality-dwi_model-csa_nodetype-parc_samples-20000streams_tracktype-local_directget-prob_minlength-20_thr-0.2.png')[0])

Above is a glass brain depiction of regions of the Harvard-Oxford atlas using a tensor model of diffusion, deterministic tractography, a minimum fiber length of 20, with 80% post-hoc thresholding using the Minimum-Spanning Tree (MST) method.
As before, node size here conveys the level of node importance (smaller is lower eigenvector centrality) and node color corresponds to hierarchical Louvain community affiliation (only two distinct communities found). Unlike in the functional case, however, edges are here depicted with dotted white lines to differentiate them from functional edges, which carry a different meaning.

In [ ]:
Image(filename=glob.glob('/Users/*/Downloads/.pynets/derivatives/pynets/sub-OAS31172/ses-d0407/dwi/atlas_harvard_oxford/figures/adjacency_OAS31172_d0407_modality-dwi_model-csa_nodetype-parc_samples-20000streams_tracktype-local_directget-det_minlength-0_thr-0.8.png')[0])


The above adjacency matrix depicts a single connectome estimate, with community affiliation. But we could also look at a structural omnetome (i.e. based on FA-weighted fiber counts) across all 72 independent connectome estimations. Note that by default pynets only samples 50,000 streamlines whose endpoints intersect with at least two parcellation regions after all tissue/waymask/minimum-length filtering. This should keep runtimes down to <2 hour for a single structural connectometry sample with comparable ensemble sampling runtimes to the functional connectometry pipeline. Bear in mind, too, that cumulatively across our ensemble here, we are *actually* sampling 50,000 x 72 = 3.6 million streamlines!

In fact if wanted, we could QC the tractogram to view these streamlines alongside the parcellation overlay(s), as is done below:

In [ ]:
%matplotlib notebook
import pathlib
from pathlib import Path
from pynets.plotting.plot_gen import view_tractogram
streams = str(Path('~').expanduser()) + '/Downloads/.pynets/derivatives/pynets/sub-OAS31172/ses-d0407/dwi/atlas_harvard_oxford/tractography/streamlines_csa_50000_parc_curv-[40_30]_step-[0.1_0.2_0.3_0.4_0.5]_dg-prob_ml-0.trk'
atlas = str(Path('~').expanduser()) + '/Downloads/.pynets/derivatives/pynets/sub-OAS31172/ses-d0407/dwi/atlas_harvard_oxford/parcellations/atlas_harvard_oxford_dwi_track.nii.gz'

#view_tractogram(streams, atlas)


Whereas in the functional connectometry case, we examined the mean connectome across estimates, here we might choose to examine the max connectome specifically, since structural connectomes are inherently sparser.

In [ ]:
%matplotlib inline
import pickle
import glob
import numpy as np
from matplotlib import pyplot as plt
from nilearn.plotting import plot_matrix
from pynets.core.thresholding import standardize, binarize

mats = [np.load(i) for i in glob.glob('/Users/*/Downloads/.pynets/derivatives/pynets/sub-OAS31172/ses-d0407/dwi/BrainnetomeAtlasFan2016/graphs/*.npy')]

max_mat = standardize(np.max(mats, axis=0))

max_mat_bin = binarize(max_mat)
max_mat = np.nan_to_num(np.multiply(max_mat, max_mat_bin))

plot_matrix(
    max_mat,
    figure=(10, 10),
    labels=[' ']*len(max_mat),
    vmax=np.percentile(max_mat[max_mat > 0], 99),
    vmin=np.percentile(max_mat[max_mat > 0], 5),
    reorder="average",
    auto_fit=True,
    grid=False,
    colorbar=False,
    cmap='gist_earth',
)
plt.show()

As you can see, we get a much more information-rich graph. This graph, unlike the first, now represents a new *distribution* of connectomes, that, by virtue of its plurality of views, more exhaustively samples from the true *population* of networks in this individual that may exhibit connectivity, across the whole brain as a region of interest.

# Collecting Outputs
Now, we explored the outputs of our connectome ensemble visually, but let's take a closer look at the actual topological data. To do this, we run another workflow using the `pynets_collect` CLI.

In [ ]:
%%bash

dir=~/Downloads/.pynets
abs_dir=`echo "$(dirname $dir)"`
pynets_collect -basedir "$abs_dir"/.pynets/derivatives/pynets -modality 'dwi'

And as before we can view the output data and visualize the various corresponding graph embeddings...

In [ ]:
import pathlib
from pathlib import Path
import pandas as pd

# Now we can load a dataframe containing all the AUC topological graph metrics calculated for this particular subject's run:
p = str(Path('~').expanduser()) + '/Downloads/.pynets/derivatives/pynets/dwi_group_topology_auc/sub-OAS31172_ses-d0407_topology_auc_clean.csv'
df_individual = pd.read_csv(p, index_col=False)
df_individual

# Note that if we were to sample connectomes from multiple subjects, the previous pynets_collect CLI would
# simply append new rows to the summary `all_subs_neat_func.csv` dataframe.
p = str(Path('~').expanduser()) + '/Downloads/.pynets/derivatives/pynets/all_subs_neat_dwi.csv'
df_group = pd.read_csv(p, index_col=False)
df_group

In [ ]:
%matplotlib inline
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
from pynets.plotting.plot_gen import plot_graph_measure_hists

csv_all_metrics = str(Path('~').expanduser()) + '/Downloads/.pynets/derivatives/pynets/dwi_group_topology_auc/sub-OAS31172_ses-d0407_topology_auc_clean.csv'

out = plot_graph_measure_hists(csv_all_metrics)
out.show()

In [ ]:
%matplotlib inline
from graspy.plot import pairplot
import glob
import os
import numpy as np
from matplotlib import pyplot as plt

omnetomes = [[np.load(i), os.path.basename(i).split('.npy')[0]] for i in glob.glob('/Users/*/Downloads/.pynets/derivatives/pynets/sub-OAS31172/ses-d0407/dwi/*/embeddings/*omnetome.npy')]

for grad, title in omnetomes:
    plot = pairplot(grad, title=title + '_Structural')

Later tutorials will cover a variety of additional topics, including how you can deploy PyNets across entire BIDS datasets in a single command-line interface (CLI) call, benchmark and optimize connectome ensembles across diverse analytic scenarios with GridSearchCV integration, along with more advanced topics such as performing and visualizing multiplex graph analysis and embeddings of multimodal connectomes.